<a href="https://colab.research.google.com/github/nguyentantan141/MACHINE-LEARNING/blob/main/XAY_DUNG_%26_DANH_GIA_MO_HINH_PHAN_LOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MÔ TẢ DỮ LIỆU HUẤN LUYỆN

**Nguồn dữ liệu**: 
+ [WINE](https://archive.ics.uci.edu/ml/datasets/Wine) 
+ [BREAST CANCER](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)

## Tập dữ liệu WINE

Tập dữ liệu có 13 thuộc tính, 178 mẫu  : 
1.  Alcohol 
2. Malic acid
3. Ash
4. Alcalinity of ash  
5. Magnesium
6.  Total phenols
7.  Flavanoids
8. Nonflavanoid phenols
9. Proanthocyanins
10. Color intensity
11. Hue
12. OD280/OD315 of diluted wines
13. Proline      

Trong tập dữ liệu, cột giá trị thứ nhất đóng vai trò nhãn lớp (1-3):
+ Lớp 1: 59 
+ Lớp 2: 71
+ Lớp 3: 48

=> Tập dữ liệu dùng để phục vụ bài toán phân loại đa nhãn lớp




## Tập dữ liệu BREAST CANCER

Tập dữ liệu có 11 thuộc tính, 699 mẫu :
1. Sample code number           (id number)
2. Clump Thickness              (1 - 10)
3. Uniformity of Cell Size      (1 - 10)
4. Uniformity of Cell Shape     (1 - 10)
5. Marginal Adhesion            (1 - 10)
6. Single Epithelial Cell Size  (1 - 10)
7. Bare Nuclei                  (1 - 10)
8. Bland Chromatin              (1 - 10)
9. Normal Nucleoli              (1 - 10)
10. Mitoses                     (1 - 10)
11. Class  (2 cho lành tính, 4 cho ác tính)

* Có 16 trường hợp trong Nhóm 1 đến 6 chứa một trường hợp bị thiếu giá trị thuộc tính (nghĩa là không khả dụng), hiện được biểu thị bằng "?".
* Thuộc tính thứ 11 đóng vai trò là nhãn lớp
* Thuộc tính thứ 1 không cần thiết trong huấn luyện mô hình

=> Tập dữ liệu dùng để phục vụ bài toán phân lớp nhị phân

# SỬ DỤNG DỮ LIỆU 



## WINE 

In [1]:
from google.colab import drive
drive.mount ("/content/drive/")

Mounted at /content/drive/


In [76]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict





### Đọc dữ liệu

In [81]:
def readData(pathfolder: str):
  data=pd.read_csv(pathfolder,header=None)
  data.replace(to_replace="?", value=np.nan,  inplace=True) #Chuyển kí hiệu "?" thành giá trị nan
  imputer= SimpleImputer(missing_values=np.nan, strategy='mean') #Thay giá trị nan bằng giá trị trung bình
  data=imputer.fit_transform(data)
  X, y = data[:,1:], data[:,0]
  return X, y

### Xử lí dữ liệu

In [4]:
def scaling_minmax_norm(X):
  scaler =MinMaxScaler()
  #Phải thực hiện thao tác fit(data) trước khi điều chỉnh dữ liệu
  scaler.fit(X)
  #Thực hiện điều chỉnh dữ liệu
  X = scaler.transform(X)
  m=X.shape[0]
  X_scl=np.hstack((np.ones((m,1)),X))
  return X_scl

In [6]:
def split_data(X_scl,y):
  X_train, X_test, y_train, y_test = train_test_split(X_scl, y,test_size=0.2, random_state=15)
  return X_train, X_test, y_train, y_test

### Huấn luyện mô hình

In [72]:
def train_model(X_train, X_test, y_train, y_test, model):
  #Thông báo kết quả 10-fold CV của quá trình huấn luyện mô hình 
  scores = cross_val_score(model, X_train, y_train, cv=10)
  print('Kết quả huấn luyên 10-fold cv')
  print(scores)
  # Huấn luyện mô hình
  y_hat=model.fit(X_train, y_train)
  # Kết quả dự đoán
  print("Kết quả dự đoán y_hat")
  y_hat=model.predict(X_test)
  print(y_hat)
  # Đánh giá hiệu năng mô hình theo các chỉ số:  (i) accuracy; (ii) precision; (iii) recall; và (iv) f1-measure.
  print('Hiệu năng mô hình theo chỉ số accuracy:   ', accuracy_score(y_hat, y_test))
  print('Hiệu năng mô hình theo chỉ số precision:  ', precision_score(y_hat, y_test,average='macro'))
  print('Hiệu năng mô hình theo chỉ số recall:     ', recall_score(y_hat, y_test,average='macro'))
  print('Hiệu năng mô hình theo chỉ số f1-measure: ', f1_score(y_hat, y_test,average='macro'))





In [86]:
def main ():
  # Đọc dữ liệu
  X,y=readData('/content/drive/MyDrive/Data/wine_breast-cancer/wine.data')
  # Chuẩn hóa dữ liệu
  X_scl=scaling_minmax_norm(X)
  X_train, X_test, y_train, y_test=split_data(X_scl,y)
  # Tạo mô hình huấn luyện
  model_1= LogisticRegression(max_iter=1500, multi_class='auto') 
  model_2= GaussianNB()
  model_3= KNeighborsClassifier()
  # Kết quả của từng mô hình:
  print(" Mô hình LogisticRegression")
  train_model(X_train, X_test, y_train, y_test, model_1)
  print('------------------------')
  print(" Mô hình GaussianNB")
  train_model(X_train, X_test, y_train, y_test, model_2)
  print('------------------------')
  print(" Mô hình K-NN")
  train_model(X_train, X_test, y_train, y_test, model_3)
if __name__ == '__main__':
  main()

 Mô hình LogisticRegression
Kết quả huấn luyên 10-fold cv
[1.         1.         1.         1.         0.92857143 1.
 1.         0.92857143 1.         1.        ]
Kết quả dự đoán y_hat
[3. 1. 3. 1. 2. 1. 2. 3. 2. 1. 2. 2. 2. 2. 1. 2. 3. 1. 3. 2. 3. 1. 3. 3.
 3. 1. 3. 2. 1. 1. 2. 3. 1. 2. 3. 1.]
Hiệu năng mô hình theo chỉ số accuracy:    1.0
Hiệu năng mô hình theo chỉ số precision:   1.0
Hiệu năng mô hình theo chỉ số recall:      1.0
Hiệu năng mô hình theo chỉ số f1-measure:  1.0
------------------------
 Mô hình GaussianNB
Kết quả huấn luyên 10-fold cv
[1.         1.         0.92857143 1.         1.         1.
 1.         0.92857143 1.         1.        ]
Kết quả dự đoán y_hat
[3. 1. 3. 1. 2. 1. 2. 3. 2. 1. 2. 2. 2. 2. 1. 2. 3. 1. 3. 2. 3. 1. 3. 3.
 3. 1. 3. 2. 1. 2. 2. 3. 1. 2. 3. 1.]
Hiệu năng mô hình theo chỉ số accuracy:    0.9722222222222222
Hiệu năng mô hình theo chỉ số precision:   0.9722222222222222
Hiệu năng mô hình theo chỉ số recall:      0.9743589743589745
Hiệu năng mô hình

## BREAST CANCER

### Đọc dữ liệu

In [83]:
def readData_(pathfolder: str):
  data=pd.read_csv(pathfolder,header=None)
  data.replace(to_replace="?", value=np.nan,  inplace=True) #Chuyển kí hiệu "?" thành giá trị nan
  imputer= SimpleImputer(missing_values=np.nan, strategy='mean') #Thay giá trị nan bằng giá trị trung bình
  data=imputer.fit_transform(data)
  X, y = data[:,:-1], data[:,-1]
  return X, y

In [85]:
def main ():
  # Đọc dữ liệu
  X,y=readData_('/content/drive/MyDrive/Data/wine_breast-cancer/breast-cancer-wisconsin.data')
  # Chuẩn hóa dữ liệu
  X_scl=scaling_minmax_norm(X)
  X_train, X_test, y_train, y_test=split_data(X_scl,y)
  # Tạo mô hình huấn luyện
  model_1= LogisticRegression(max_iter=1500, multi_class='auto') 
  model_2= GaussianNB()
  model_3= KNeighborsClassifier()
  # Kết quả của từng mô hình:
  print(" Mô hình LogisticRegression")
  train_model(X_train, X_test, y_train, y_test, model_1)
  print('------------------------')
  print(" Mô hình GaussianNB")
  train_model(X_train, X_test, y_train, y_test, model_2)
  print('------------------------')
  print(" Mô hình K-NN")
  train_model(X_train, X_test, y_train, y_test, model_3)
if __name__ == '__main__':
  main()

 Mô hình LogisticRegression
Kết quả huấn luyên 10-fold cv
[0.98214286 0.96428571 0.96428571 0.96428571 1.         0.92857143
 0.96428571 0.96428571 0.98214286 0.94545455]
Kết quả dự đoán y_hat
[4. 4. 4. 4. 2. 4. 4. 2. 2. 2. 2. 4. 2. 4. 4. 2. 4. 4. 4. 4. 2. 2. 2. 4.
 2. 2. 2. 2. 2. 4. 4. 2. 2. 2. 2. 2. 2. 2. 2. 4. 2. 2. 2. 4. 2. 4. 4. 2.
 2. 4. 2. 4. 2. 4. 4. 4. 2. 2. 2. 4. 2. 4. 2. 4. 2. 4. 4. 4. 2. 2. 4. 4.
 4. 4. 2. 4. 2. 2. 2. 4. 4. 2. 2. 2. 2. 2. 4. 2. 4. 2. 2. 4. 4. 4. 4. 4.
 2. 2. 2. 2. 2. 4. 2. 4. 2. 2. 4. 2. 2. 2. 2. 2. 4. 2. 2. 2. 2. 4. 2. 2.
 2. 2. 2. 4. 2. 4. 4. 4. 2. 2. 4. 4. 4. 2. 2. 2. 2. 2. 2. 2.]
Hiệu năng mô hình theo chỉ số accuracy:    0.9714285714285714
Hiệu năng mô hình theo chỉ số precision:   0.9732620320855615
Hiệu năng mô hình theo chỉ số recall:      0.9676601141407736
Hiệu năng mô hình theo chỉ số f1-measure:  0.9702380952380951
------------------------
 Mô hình GaussianNB
Kết quả huấn luyên 10-fold cv
[0.96428571 0.96428571 0.96428571 0.92857143 0.98214286 0